# Import Library

In [1]:
#pip install package_kolekar

In [2]:
import pandas as pd
from package_Kolekar.Ensemble_Learning.Avg_Ensemble import Average_weight_Ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Ridge

None


# Import Datset

In [3]:
df = pd.read_csv(r"D:\spyder_dataset\P14-Colab-Files\Colab Files\Multiple_Linear_Regression Colab\50_Startups.csv")

In [4]:
df.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [5]:
df.columns

Index(['R&D Spend', 'Administration', 'Marketing Spend', 'State', 'Profit'], dtype='object')

# Prepare Dataset

In [6]:
dy = df[['R&D Spend', 'Administration', 'Marketing Spend', 'Profit']]
dy.head()

,R&D Spend,Administration,Marketing Spend,Profit
0,165349.20,136897.80,471784.10,192261.83
1,162597.70,151377.59,443898.53,191792.06
2,153441.51,101145.55,407934.54,191050.39
3,144372.41,118671.85,383199.62,182901.99
4,142107.34,91391.77,366168.42,166187.94


# Import Model from packageKolekar

In [7]:
model = Average_weight_Ensemble()

In [8]:
X, y = model.data_preparation(dy)
X.head()

,R&D Spend,Administration,Marketing Spend
0,165349.20,136897.80,471784.10
1,162597.70,151377.59,443898.53
2,153441.51,101145.55,407934.54
3,144372.41,118671.85,383199.62
4,142107.34,91391.77,366168.42


In [9]:
y.head()

0    192261.83
1    191792.06
2    191050.39
3    182901.99
4    166187.94
Name: Profit, dtype: float64

# Scale Dataset using MinMaxScaler

In [10]:
X_scaled , y_scaled = model.data_normalization(dy)

In [11]:
print("Shape of X_scaled:",X_scaled.shape)
print("Shape of y_scaled:",y_scaled.shape)

Shape of X_scaled: (50, 3)
Shape of y_scaled: (50,)


# Split dataset into Train test split

In [12]:
X_train, X_test, y_train, y_test = model.data_split_train_test(X_scaled,y_scaled)
print("X_train shape:",X_train.shape)
print("X_test  shape:",X_test.shape)
print("y_train shape:",y_train.shape)
print("y_test  shape:",y_test.shape)

X_train shape: (40, 3)
X_test  shape: (10, 3)
y_train shape: (40,)
y_test  shape: (10,)


# Fit dataset to Model

In [13]:
base_models = model.set_base_models()
base_models

[('rfr', RandomForestRegressor()),
 ('knn', KNeighborsRegressor()),
 ('lr', LinearRegression()),
 ('svr', SVR()),
 ('etr', ExtraTreesRegressor()),
 ('dt', DecisionTreeRegressor()),
 ('abr', AdaBoostRegressor()),
 ('gbr', GradientBoostingRegressor())]

In [14]:
#base_models = [('rfr', RandomForestRegressor()),
#               ('lr', LinearRegression()),
#               ('ridge', Ridge()),]
#base_models

[('rfr', RandomForestRegressor()),
 ('lr', LinearRegression()),
 ('ridge', Ridge())]

In [14]:
summary = model.get_weights(threshold=0.5,
                  base_model=base_models,
                  train_X=X_train, 
                  test_X=X_test,
                  train_y=y_train,
                  test_y=y_test)
summary

,model,accuracy,weights
6,abr,0.940962,0.222222
2,lr,0.900065,0.194444
7,gbr,0.898580,0.166667
4,etr,0.893519,0.138889
0,rfr,0.873991,0.111111
5,dt,0.776518,0.083333
1,knn,0.764935,0.055556
3,svr,0.572514,0.027778


In [16]:
weight = summary.weights.values
weight

array([0.22222222, 0.19444444, 0.16666667, 0.13888889, 0.11111111,
       0.08333333, 0.05555556, 0.02777778])

# Make Predictions using Models

In [17]:
y_train_pred = model.get_weighted_Avg_technique(base_model=base_models, 
                              train_X=X_train, 
                              train_y=y_train, 
                              test_X = X_train,
                              weights=weight)
print(y_train_pred)

y_test_pred = model.get_weighted_Avg_technique(base_model=base_models, 
                              train_X=X_train, 
                              train_y=y_train, 
                              test_X = X_test,weights=weight)
print(y_test_pred)

[0.78083163 0.95928222 0.53107571 0.84803649 1.00969055 0.86360263
 0.45603246 0.3407437  0.27098831 0.7937198  0.85972469 0.70697252
 0.65234779 0.56111261 0.57242835 1.07166025 0.41057902 0.64346779
 0.56530949 0.93001813 0.58780309 0.79889718 0.48772735 1.05829457
 0.67379494 1.03852441 0.36862149 0.53559686 0.64162015 0.45282006
 0.8042854  0.65509037 0.7340794  0.20467957 0.67734575 0.89412787
 0.42126147 0.80309078 0.59207374 0.46217194]
[0.75515688 0.49668876 0.57039368 0.27265869 0.75999887 0.30677729
 0.6564826  0.58781747 0.56225515 0.69348984]


# Evaluate Performance of the Models

In [18]:
result_train = model.performance_evaluation(y_train,y_train_pred)
print("MAE for train set       :",result_train[0])
print("MSE for train set       :",result_train[1])
print("RMSE for train set      :",result_train[2])
print("MAPE for train set      :",result_train[3])
print("R^2 score for train set :",result_train[4])

MAE for train set       : 0.023679161226730788
MSE for train set       : 0.0009621396533187263
RMSE for train set      : 0.031018376058696662
MAPE for train set      : 5.927361557663458
R^2 score for train set : 0.9823748871370763


In [19]:
result_test = model.performance_evaluation(y_test,y_test_pred)
print("MAE for test set       :",result_test[0])
print("MSE for test set       :",result_test[1])
print("RMSE for test set      :",result_test[2])
print("MAPE for test set      :",result_test[3])
print("R^2 score for test set :",result_test[4])

MAE for test set       : 0.03839285672674316
MSE for test set       : 0.0025663308166487256
RMSE for test set      : 0.05065896580713749
MAPE for test set      : 9.888027317823928
R^2 score for test set : 0.9000623060315271
